In [47]:
!pip install --upgrade --quiet pip
!pip install --upgrade --quiet datasets[audio] transformers accelerate evaluate jiwer tensorboard gradio

In [2]:
import pandas as pd
from huggingface_hub import notebook_login
# from datasets import load_dataset, DatasetDict
from sklearn.model_selection import train_test_split

In [3]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

/bin/bash: line 1: nvidia-smi: command not found


In [4]:
TRANSCRIPTION_CSV = "https://raw.githubusercontent.com/AI-Lab-Makerere/Yogera-Dataset-Metadata/main/version_4.0.1/nyn/transcribed_and_reviewed.csv"
transcription_df = pd.read_csv(TRANSCRIPTION_CSV)
# transcription_df['path'] = transcription_df.voice_clip.apply(lambda x: 'audio/'+str(x))
transcription_df['link'] = transcription_df.voice_clip.apply(lambda x: 'https://storage.googleapis.com/yogera_voices_backup/Runyankole-Rukiga/'+str(x))
transcription_df.head(4)

,phase,image_url,contributor_id,gender,age_group,language,voice_clip,duration,up_votes,down_votes,transcript,link
0,1.1,https://storage.googleapis.com/image_prompts/4...,655,Male,18-29,Runyankole-Rukiga,yogera_image_audio_20240114_025705.520475_1334...,0.007778,3,0,Enju ibiri z'obudongo emwe eshakize ebinyatsi ...,https://storage.googleapis.com/yogera_voices_b...
1,1.1,https://storage.googleapis.com/image_prompts/5...,655,Male,18-29,Runyankole-Rukiga,yogera_image_audio_20240112_190942.084373_1843...,0.004722,3,0,Abantu aba bari omu bwato bwaabo bari omu maiz...,https://storage.googleapis.com/yogera_voices_b...
2,1.1,https://storage.googleapis.com/image_prompts/u...,655,Male,18-29,Runyankole-Rukiga,yogera_image_audio_20240111_202511.881017_2974...,0.006944,3,0,Omushaija ashutami aha ntebe aineho laputopu y...,https://storage.googleapis.com/yogera_voices_b...
3,1.1,https://storage.googleapis.com/image_prompts/7...,655,Male,18-29,Runyankole-Rukiga,yogera_image_audio_20240111_203224.142845_2290...,0.007778,3,0,Omukazi ari aha rubaju rw'eduuka Ari omu muhan...,https://storage.googleapis.com/yogera_voices_b...


In [5]:
transcription_df.duration.sum()

30.074722222222015

In [44]:
from huggingface_hub import HfApi
api = HfApi()
from google.colab import userdata
my_repo_id = "Shawal777/yogera_runyankore_ailab_4_0_1"
api.create_repo(
    repo_id = "Shawal777/yogera_runyankore_ailab_4_0_1",
    exist_ok = True,
    token=userdata.get('HF_TOKEN'),
    repo_type = "dataset"
)
# api.upload_folder(
#     folder_path="audio",
#     repo_id="Shawal777/yogeraRunyankore",
#     repo_type="dataset",
#     token=userdata.get('HF_TOKEN')
# )

RepoUrl('https://huggingface.co/datasets/Shawal777/yogera_runyankore_ailab_4_0_1', endpoint='https://huggingface.co', repo_type='dataset', repo_id='Shawal777/yogera_runyankore_ailab_4_0_1')

In [37]:
cid_hr = transcription_df.groupby(transcription_df.contributor_id).duration.sum()
test_cid = pd.DataFrame(cid_hr[cid_hr.cumsum()>27])

c = list(test_cid.index)

train_df = transcription_df[~transcription_df.contributor_id.isin(c)]
test_df = transcription_df[transcription_df.contributor_id.isin(c)]
'train hours', train_df.duration.sum(), 'test hours', test_df.duration.sum()

('train hours', 26.833333333333144, 'test hours', 3.2413888888888684)

In [42]:
train_df.contributor_id.unique()

array([ 655,  645,  553,  556,  559,  605,  651,  678,  552,  570,  539,
        681,  609,  572,   30,   48,   98,   88,  108,   85,   89,   93,
         80,  106,   86,   99,   81,   84,  114,  122,  123,  113,  125,
        129,  127,  118,  142,  136,  132,  140,  117,  135,  137,  120,
       1343,  849,   82,  121,  863,  116,   25,   34,   37,   46,  105,
        103,  112,   96,   94,   90,  131,  126,  115,  104,  128,   95,
        555,  537,  571,  755,  610,  847,  868,  766, 1447,  759,  769,
        873,  750,  764,  862,  753,  763,  888,  761,  762,  757,  758,
        787,  749, 1516,  767,  756,  752,  765,  879,  848,  768,  134,
         92, 1476,  857, 1060,  878,  770, 1119, 1159,  521, 1120, 1124,
       1497, 1179, 1099, 1157, 1195, 1217, 1250, 1131, 1245, 1262, 1266,
       1143, 1520, 1158, 1201, 1222, 1218, 1187,   97,  440, 1438,  100,
       1382,  143,  124, 1289, 1324, 1292, 1389, 1402, 1414, 1503, 1541,
       1339, 1699, 1295, 1424, 1505, 1611, 1692, 13

In [41]:
test_df.contributor_id.unique()

array([1763, 1754, 1760, 1794, 1797, 1789, 1773, 1761, 1804, 1792, 1817,
       1762, 1770, 1755, 1843, 1830, 1868, 1862, 1872, 1874, 1882, 1870,
       1883, 1759, 1887, 1839, 1903, 1915, 1967, 1984, 1802, 2001, 2059,
       2072, 2101, 1803, 1886, 1798, 2112])

In [39]:
train_records = train_df.to_dict(orient='records')
test_records = test_df.to_dict(orient='records')

import json
with open('train.jsonl','w') as file:
    for line in train_records:
        file.write(json.dumps(line) +'\n\r')
with open('test.jsonl','w') as file:
    for line in test_records:
        file.write(json.dumps(line) +'\n\r')
'train',len(train_records),'test', len(test_records)

('train', 4628, 'test', 507)

In [45]:
api.upload_file(
    path_or_fileobj='train.jsonl',
    repo_id=my_repo_id,
    path_in_repo='train.jsonl',
    repo_type="dataset",
    token=userdata.get('HF_TOKEN')
)
api.upload_file(
    path_or_fileobj='test.jsonl',
    repo_id=my_repo_id,
    path_in_repo='test.jsonl',
    repo_type="dataset",
    token=userdata.get('HF_TOKEN')
)

CommitInfo(commit_url='https://huggingface.co/datasets/Shawal777/yogera_runyankore_ailab_4_0_1/commit/f27bfe2fcd3ed6705bf09801454b729190ac2997', commit_message='Upload test.jsonl with huggingface_hub', commit_description='', oid='f27bfe2fcd3ed6705bf09801454b729190ac2997', pr_url=None, pr_revision=None, pr_num=None)

In [50]:
from datasets import DatasetDict, load_dataset
runyankore = DatasetDict()
runyankore = load_dataset(my_repo_id)
runyankore

Generating train split:   0%|          | 0/4628 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/507 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['phase', 'image_url', 'contributor_id', 'gender', 'age_group', 'language', 'voice_clip', 'duration', 'up_votes', 'down_votes', 'transcript', 'link'],
        num_rows: 4628
    })
    test: Dataset({
        features: ['phase', 'image_url', 'contributor_id', 'gender', 'age_group', 'language', 'voice_clip', 'duration', 'up_votes', 'down_votes', 'transcript', 'link'],
        num_rows: 507
    })
})